<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-phoenix-assets/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://arize-ai.slack.com/join/shared_invite/zt-2w57bhem8-hq24MB6u7yE_ZF_ilOYSBw#/shared-invite/email">Community</a>
    </p>
</center>
<h1 align="center">Mistral: Evaluate RAG with LLM Evals</h1>

In this tutorial we will look into building a RAG pipeline and evaluating it with Phoenix Evals.

It has the the following sections:

1. Understanding Retrieval Augmented Generation (RAG).
2. Building RAG (with the help of a framework such as LlamaIndex and LLM providers like Mistral).
3. Evaluating RAG with Evals.

## Retrieval Augmented Generation (RAG)

LLMs are trained on vast datasets, but these will not include your specific data (things like company knowledge bases and documentation). Retrieval-Augmented Generation (RAG) addresses this by dynamically incorporating your data as context during the generation process. This is done not by altering the training data of the LLMs but by allowing the model to access and utilize your data in real-time to provide more tailored and contextually relevant responses.

In RAG, your data is loaded and prepared for queries. This process is called indexing. User queries act on this index, which filters your data down to the most relevant context. This context and your query then are sent to the LLM along with a prompt, and the LLM provides a response.

RAG is a critical component for building applications such a chatbots or agents and you will want to know RAG techniques on how to get data into your application.

<img src="https://storage.googleapis.com/arize-phoenix-assets/assets/images/RAG_Pipeline.png" width="800px">

## Stages within RAG

There are five key stages within RAG, which will in turn be a part of any larger RAG application.

- **Loading**: This refers to getting your data from where it lives - whether it's text files, PDFs, another website, a database or an API - into your pipeline.
- **Indexing**: This means creating a data structure that allows for querying the data. For LLMs this nearly always means creating vector embeddings, numerical representations of the meaning of your data, as well as numerous other metadata strategies to make it easy to accurately find contextually relevant data.
- **Storing**: Once your data is indexed, you will want to store your index, along with any other metadata, to avoid the need to re-index it.

- **Querying**: For any given indexing strategy there are many ways you can utilize LLMs and data structures to query, including sub-queries, multi-step queries, and hybrid strategies. 
- **Evaluation**: A critical step in any pipeline is checking how effective it is relative to other strategies, or when you make changes. Evaluation provides objective measures on how accurate, faithful, and fast your responses to queries are.


## Build a RAG system 

Now that we have understood the stages of RAG, let's build a pipeline. We will use [LlamaIndex](https://www.llamaindex.ai/) for RAG and [Phoenix Evals](https://docs.arize.com/phoenix/llm-evals/llm-evals) for evaluation.


In [ ]:
%pip install -Uqq "arize-phoenix[embeddings]" "arize-phoenix-evals>=0.5.0" "llama-index==0.10.19" "llama-index-llms-mistralai" "llama-index-embeddings-mistralai"  "openinference-instrumentation-mistralai>=0.0.2" "openinference-instrumentation-llama-index>=1.0.0" "llama-index-callbacks-arize-phoenix>=0.1.2" gcsfs nest_asyncio 'httpx<0.28'

First, enter your Mistral API key. Mistral will be used for indexing, querying, and evaluation.

In [ ]:
import os
from getpass import getpass

if not (mistral_api_key := os.getenv("MISTRAL_API_KEY")):
    mistral_api_key = getpass("🔑 Enter your MISTRAL API key: ")
os.environ["MISTRAL_API_KEY"] = mistral_api_key

Let's setup our environment and a set of constants to use throughout the tutorial.

In [ ]:
import os

# The nest_asyncio module enables the nesting of asynchronous functions within an already running async loop.
# This is necessary because Jupyter notebooks inherently operate in an asynchronous loop.
# By applying nest_asyncio, we can run additional async functions within this existing loop without conflicts.
import nest_asyncio

# Setup projects to collect tracing under
os.environ["PHOENIX_PROJECT_NAME"] = "mistral-rag"  # Collect traces under the project "mistral-rag"
INDEXING_PROJECT = "indexing"  # For llama-index indexing
TESTSET_PROJECT = "testset"  # For capturing synthetic testset traces
MAX_QUERIES = 10  # Maximum number of queries to run.

nest_asyncio.apply()

In [ ]:
import phoenix as px

(session := px.launch_app()).view()

Let's use the OpenInference Mistral Instrumentation as well.

In [ ]:
from openinference.instrumentation.llama_index import LlamaIndexInstrumentor
from openinference.instrumentation.mistralai import MistralAIInstrumentor

from phoenix.otel import register

tracer_provider = register(endpoint="http://127.0.0.1:6006/v1/traces")
MistralAIInstrumentor().instrument(skip_dep_check=True, tracer_provider=tracer_provider)
LlamaIndexInstrumentor().instrument(skip_dep_check=True, tracer_provider=tracer_provider)

In [ ]:
import pandas as pd
from llama_index.core import Settings, SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.embeddings.mistralai import MistralAIEmbedding
from llama_index.llms.mistralai import MistralAI

import phoenix as px
from phoenix.trace import using_project

During this tutorial, we will capture all the data we need to evaluate our RAG pipeline using Phoenix Tracing. To enable this, simply start the phoenix application and instrument LlamaIndex.

In [ ]:
(session := px.launch_app()).view()

Enable Phoenix tracing via `LlamaIndexInstrumentor`. Phoenix uses OpenInference traces - an open-source standard for capturing and storing LLM application traces that enables LLM applications to seamlessly integrate with LLM observability solutions such as Phoenix.

### Load Data and Build an Index

Let's use an [essay by Paul Graham](https://www.paulgraham.com/worked.html) to build our RAG pipeline.

In [ ]:
import tempfile
from urllib.request import urlretrieve

with tempfile.NamedTemporaryFile() as tf:
    urlretrieve(
        "https://raw.githubusercontent.com/Arize-ai/phoenix-assets/main/data/paul_graham/paul_graham_essay.txt",
        tf.name,
    )
    documents = SimpleDirectoryReader(input_files=[tf.name]).load_data()

In [ ]:
# Define an LLM
llm = MistralAI(model="mistral-large-latest")
Settings.llm = llm
Settings.embed_model = MistralAIEmbedding()

with using_project(INDEXING_PROJECT):  # Collect traces under the project "indexing"
    # Build index with a chunk_size of 512
    node_parser = SimpleNodeParser.from_defaults(chunk_size=512)
    nodes = node_parser.get_nodes_from_documents(documents)
    vector_index = VectorStoreIndex(nodes)

Build a QueryEngine and start querying.

In [ ]:
query_engine = vector_index.as_query_engine()

In [ ]:
response_vector = query_engine.query("What did the author do growing up?")

Check the response that you get from the query.

In [ ]:
response_vector.response

By default LlamaIndex retrieves two similar nodes/ chunks. You can modify that in `vector_index.as_query_engine(similarity_top_k=k)`.

Let's check the text in each of these retrieved nodes.

In [ ]:
# First retrieved node
response_vector.source_nodes[0].get_text()

In [ ]:
# Second retrieved node
response_vector.source_nodes[1].get_text()

Remember that we are using Phoenix Tracing to capture all the data we need to evaluate our RAG pipeline. You can view the traces in the phoenix application.

In [ ]:
print("phoenix URL", px.active_session().url)

We can access the traces by directly pulling the spans from the phoenix session.

In [ ]:
spans_df = px.Client().get_spans_dataframe()

In [ ]:
spans_df[["name", "span_kind", "attributes.input.value", "attributes.retrieval.documents"]].head()

Note that the traces have captured the documents that were retrieved by the query engine. This is nice because it means we can introspect the documents without having to keep track of them ourselves.

In [ ]:
spans_with_docs_df = spans_df[spans_df["attributes.retrieval.documents"].notnull()]

In [ ]:
spans_with_docs_df[["attributes.input.value", "attributes.retrieval.documents"]].head()

We have built a RAG pipeline and also have instrumented it using Phoenix Tracing. We now need to evaluate it's performance. We can assess our RAG system/query engine using Phoenix's LLM Evals. Let's examine how to leverage these tools to quantify the quality of our retrieval-augmented generation system.

## Evaluation

Evaluation should serve as the primary metric for assessing your RAG application. It determines whether the pipeline will produce accurate responses based on the data sources and range of queries.

While it's beneficial to examine individual queries and responses, this approach is impractical as the volume of edge-cases and failures increases. Instead, it's more effective to establish a suite of metrics and automated evaluations. These tools can provide insights into overall system performance and can identify specific areas that may require scrutiny.

In a RAG system, evaluation focuses on two critical aspects:

- **Retrieval Evaluation**: To assess the accuracy and relevance of the documents that were retrieved
- **Response Evaluation**: Measure the appropriateness of the response generated by the system when the context was provided.

### Generate Question Context Pairs

For the evaluation of a RAG system, it's essential to have queries that can fetch the correct context and subsequently generate an appropriate response.

For this tutorial, let's use Phoenix's `llm_generate` to help us create the question-context pairs.

First, let's create a dataframe of all the document chunks that we have indexed.

In [ ]:
# Let's construct a dataframe of just the documents that are in our index
document_chunks_df = pd.DataFrame({"text": [node.get_text() for node in nodes]})
document_chunks_df.head()

Now that we have the document chunks, let's prompt an LLM to generate us 3 questions per chunk. Note that you could manually solicit questions from your team or customers, but this is a quick and easy way to generate a large number of questions.

In [ ]:
generate_questions_template = """\
Context information is below.

---------------------
{text}
---------------------

Given the context information and not prior knowledge.
generate only questions based on the below query.

You are a Teacher/ Professor. Your task is to setup \
3 questions for an upcoming \
quiz/examination. The questions should be diverse in nature \
across the document. Restrict the questions to the \
context information provided."

Output the questions in JSON format with the keys question_1, question_2, question_3.
"""

In [ ]:
import json

from phoenix.evals import MistralAIModel, llm_generate


def output_parser(response: str, index: int):
    try:
        return json.loads(response)
    except json.JSONDecodeError as e:
        return {"__error__": str(e)}


with using_project(TESTSET_PROJECT):  # Collect traces under the project "testset"
    questions_df = llm_generate(
        dataframe=document_chunks_df,
        template=generate_questions_template,
        model=MistralAIModel(model="mistral-large-latest", response_format={"type": "json_object"}),
        output_parser=output_parser,
        concurrency=20,
    )

MistralAIInstrumentor().uninstrument()  # No longer needed

In [ ]:
questions_df.head()

In [ ]:
# Construct a dataframe of the questions and the document chunks
questions_with_document_chunk_df = pd.concat([questions_df, document_chunks_df], axis=1)
questions_with_document_chunk_df = questions_with_document_chunk_df.melt(
    id_vars=["text"], value_name="question"
).drop("variable", axis=1)
# If the above step was interrupted, there might be questions missing. Let's run this to clean up the dataframe.
questions_with_document_chunk_df = questions_with_document_chunk_df[
    questions_with_document_chunk_df["question"].notnull()
]

The LLM has generated three questions per chunk. Let's take a quick look.

In [ ]:
questions_with_document_chunk_df.head(10)

### Retrieval Evaluation

We are now prepared to perform our retrieval evaluations. We will execute the queries we generated in the previous step and verify whether or not that the correct context is retrieved.

In [ ]:
# loop over the questions and generate the answers
for index, row in questions_with_document_chunk_df.iterrows():
    if index >= MAX_QUERIES:
        break
    question = row["question"]
    response_vector = query_engine.query(question)
    print(f"Question: {question}\nAnswer: {response_vector.response}\n")

Now that we have executed the queries, we can start validating whether or not the RAG system was able to retrieve the correct context. Let's extract all the retrieved documents from the traces logged to phoenix. (For an in-depth explanation of how to export trace data from the phoenix runtime, consult the [docs](https://docs.arize.com/phoenix/how-to/extract-data-from-spans)).

In [ ]:
from phoenix.session.evaluation import get_retrieved_documents

retrieved_documents_df = get_retrieved_documents(px.Client())
retrieved_documents_df

Let's now use Phoenix's LLM Evals to evaluate the relevance of the retrieved documents with regards to the query. Note, we've turned on `explanations` which prompts the LLM to explain it's reasoning. This can be useful for debugging and for figuring out potential corrective actions.

In [ ]:
from phoenix.evals import (
    RelevanceEvaluator,
    run_evals,
)

relevance_evaluator = RelevanceEvaluator(MistralAIModel(model="mistral-large-latest"))

retrieved_documents_relevance_df = run_evals(
    evaluators=[relevance_evaluator],
    dataframe=retrieved_documents_df,
    provide_explanation=True,
    concurrency=20,
)[0]

In [ ]:
retrieved_documents_relevance_df.head()

We can now combine the documents with the relevance evaluations to compute retrieval metrics. These metrics will help us understand how well the RAG system is performing.

In [ ]:
documents_with_relevance_df = pd.concat(
    [retrieved_documents_df, retrieved_documents_relevance_df.add_prefix("eval_")], axis=1
)
documents_with_relevance_df

Let's compute Normalized Discounted Cumulative Gain [NCDG](https://en.wikipedia.org/wiki/Discounted_cumulative_gain) at 2 for all our retrieval steps.  In information retrieval, this metric is often used to measure effectiveness of search engine algorithms and related applications.

In [ ]:
import numpy as np
from sklearn.metrics import ndcg_score


def _compute_ndcg(df: pd.DataFrame, k: int):
    """Compute NDCG@k in the presence of missing values"""
    n = max(2, len(df))
    eval_scores = np.zeros(n)
    doc_scores = np.zeros(n)
    eval_scores[: len(df)] = df.eval_score
    doc_scores[: len(df)] = df.document_score
    try:
        return ndcg_score([eval_scores], [doc_scores], k=k)
    except ValueError:
        return np.nan


ndcg_at_2 = pd.DataFrame(
    {"score": documents_with_relevance_df.groupby("context.span_id").apply(_compute_ndcg, k=2)}
)

In [ ]:
ndcg_at_2

Let's also compute precision at 2 for all our retrieval steps.

In [ ]:
precision_at_2 = pd.DataFrame(
    {
        "score": documents_with_relevance_df.groupby("context.span_id").apply(
            lambda x: x.eval_score[:2].sum(skipna=False) / 2
        )
    }
)

In [ ]:
precision_at_2

Lastly, let's compute whether or not a correct document was retrieved at all for each query (e.g. a hit)

In [ ]:
hit = pd.DataFrame(
    {
        "hit": documents_with_relevance_df.groupby("context.span_id").apply(
            lambda x: x.eval_score[:2].sum(skipna=False) > 0
        )
    }
)

Let's now view the results in a combined dataframe.

In [ ]:
retrievals_df = px.Client().get_spans_dataframe("span_kind == 'RETRIEVER'")
rag_evaluation_dataframe = pd.concat(
    [
        retrievals_df["attributes.input.value"],
        ndcg_at_2.add_prefix("ncdg@2_"),
        precision_at_2.add_prefix("precision@2_"),
        hit,
    ],
    axis=1,
)
rag_evaluation_dataframe

### Observations

Let's now take our results and aggregate them to get a sense of how well our RAG system is performing.

In [ ]:
# Aggregate the scores across the retrievals
results = rag_evaluation_dataframe.mean(numeric_only=True)
results

As we can see from the above numbers, our RAG system is not perfect, there are times when it fails to retrieve the correct context within the first two documents. At other times the correct context is included in the top 2 results but non-relevant information is also included in the context. This is an indication that we need to improve our retrieval strategy. One possible solution could be to increase the number of documents retrieved and then use a more sophisticated ranking strategy (such as a reranker) to select the correct context.

We have now evaluated our RAG system's retrieval performance. Let's send these evaluations to Phoenix for visualization. By sending the evaluations to Phoenix, you will be able to view the evaluations alongside the traces that were captured earlier.

In [ ]:
from phoenix.trace import DocumentEvaluations, SpanEvaluations

px.Client().log_evaluations(
    SpanEvaluations(dataframe=ndcg_at_2, eval_name="ndcg@2"),
    SpanEvaluations(dataframe=precision_at_2, eval_name="precision@2"),
    DocumentEvaluations(dataframe=retrieved_documents_relevance_df, eval_name="relevance"),
)

In [ ]:
ndcg_at_2.head()

### Response Evaluation

The retrieval evaluations demonstrates that our RAG system is not perfect. However, it's possible that the LLM is able to generate the correct response even when the context is incorrect. Let's evaluate the responses generated by the LLM.

In [ ]:
from phoenix.session.evaluation import get_qa_with_reference

qa_with_reference_df = get_qa_with_reference(px.Client())
qa_with_reference_df

Now that we have a dataset of the question, context, and response (input, reference, and output), we now can measure how well the LLM is responding to the queries. For details on the QA correctness evaluation, see the [LLM Evals documentation](https://docs.arize.com/phoenix/llm-evals/running-pre-tested-evals/q-and-a-on-retrieved-data).

In [ ]:
from phoenix.evals import (
    HallucinationEvaluator,
    MistralAIModel,
    QAEvaluator,
    run_evals,
)

qa_evaluator = QAEvaluator(MistralAIModel(model="mistral-large-latest"))
hallucination_evaluator = HallucinationEvaluator(MistralAIModel(model="mistral-large-latest"))

qa_correctness_eval_df, hallucination_eval_df = run_evals(
    evaluators=[qa_evaluator, hallucination_evaluator],
    dataframe=qa_with_reference_df,
    provide_explanation=True,
    concurrency=20,
)

In [ ]:
qa_correctness_eval_df.head()

In [ ]:
hallucination_eval_df.head()

#### Observations

Let's now take our results and aggregate them to get a sense of how well the LLM is answering the questions given the context.

In [ ]:
qa_correctness_eval_df.mean(numeric_only=True)

In [ ]:
hallucination_eval_df.mean(numeric_only=True)

Our QA Correctness score of `0.91` and a Hallucinations score `0.05` signifies that the generated answers are correct ~91% of the time and that the responses contain hallucinations 5% of the time - there is room for improvement. This could be due to the retrieval strategy or the LLM itself. We will need to investigate further to determine the root cause.

Since we have evaluated our RAG system's QA performance and Hallucinations performance, let's send these evaluations to Phoenix for visualization.

In [ ]:
from phoenix.trace import SpanEvaluations

px.Client().log_evaluations(
    SpanEvaluations(dataframe=qa_correctness_eval_df, eval_name="Q&A Correctness"),
    SpanEvaluations(dataframe=hallucination_eval_df, eval_name="Hallucination"),
)

We now have sent all our evaluations to Phoenix. Let's go to the Phoenix application and view the results! Since we've sent all the evals to Phoenix, we can analyze the results together to make a determination on whether or not poor retrieval or irrelevant context has an effect on the LLM's ability to generate the correct response.

In [ ]:
print("phoenix URL", px.active_session().url)

## Embeddings Analysis
[Embeddings](https://arize.com/blog-course/embeddings-meaning-examples-and-how-to-compute/) encode the meaning of retrieved documents and user queries. Not only are they an essential part of RAG systems, but they are immensely useful for understanding and debugging LLM application performance.

Phoenix takes the high-dimensional embeddings from your RAG application, reduces their dimensionality, and clusters them into semantically meaningful groups of data. You can then select the metric of your choice (e.g. hallucinations or QA correctness) to visually inspect the performance of your application and surface problematic clusters. The advantage of this approach is that it provides metrics on granular yet meaningful subsets of your data that help you analyze local, not merely global, performance across a dataset. It's also helpful for gaining intuition around what kind of queries your LLM application is struggling to answer.

In [ ]:
# First, let's grab the embeddings from our queries
from phoenix.trace.dsl.helpers import SpanQuery

embeddings_df = px.Client().query_spans(
    SpanQuery()
    .with_index("trace_id")
    .explode(
        "embedding.embeddings",
        query="embedding.text",
        vector="embedding.vector",
    ),
)
queries_df = px.Client().query_spans(
    SpanQuery()
    .with_index("trace_id")
    .select("span_id", response="output.value")
    .where("parent_id is None")
)
query_embeddings_df = queries_df.join(embeddings_df, how="inner").set_index("context.span_id")
query_embeddings_df.head()

In [ ]:
# Now let's add our evaluations to the dataframe
query_embeddings_with_evals_df = pd.concat(
    [
        hallucination_eval_df[["label", "score"]].rename(
            columns={"label": "hallucination_label", "score": "hallucination_score"}
        ),
        qa_correctness_eval_df[["label", "score"]].rename(
            columns={"label": "qa_correctness_label", "score": "qa_correctness_score"}
        ),
        query_embeddings_df,
    ],
    axis=1,  # joining on the row indices
    join="inner",  # inner-join by the indices of the DataFrames
)

query_embeddings_with_evals_df.head()

In [ ]:
# Next let's grab the embeddings from our corpus (indexing)
from phoenix.trace.dsl.helpers import SpanQuery

client = px.Client()
corpus_df = px.Client().query_spans(
    SpanQuery().explode(
        "embedding.embeddings",
        text="embedding.text",
        vector="embedding.vector",
    ),
    project_name=INDEXING_PROJECT,
)
corpus_df.head()

In [ ]:
# Let's now merge in our evaluations into the queries
query_embeddings_df = query_embeddings_df.iloc[::-1]
query_df = pd.concat(
    [
        hallucination_eval_df[["score", "label"]]
        .rename(columns={"score": "hallucination_score", "label": "hallucination_label"})
        .reset_index(drop=True),
        qa_correctness_eval_df[["score", "label"]]
        .rename(columns={"score": "qa_correctness_score", "label": "qa_correctness_label"})
        .reset_index(drop=True),
        query_embeddings_df[["query", "response", "vector"]].reset_index(drop=True),
    ],
    axis=1,
)
query_df.head()

In [ ]:
query_schema = px.Schema(
    prompt_column_names=px.EmbeddingColumnNames(
        raw_data_column_name="query", vector_column_name="vector"
    ),
    response_column_names="response",
    tag_column_names=[
        "hallucination_label",
        "hallucination_score",
        "qa_correctness_label",
        "qa_correctness_score",
    ],
)
corpus_schema = px.Schema(
    prompt_column_names=px.EmbeddingColumnNames(
        raw_data_column_name="text", vector_column_name="vector"
    )
)
# relaunch phoenix with a primary and corpus dataset to view embeddings
px.close_app()
session = px.launch_app(
    primary=px.Inferences(query_df, query_schema, "query"),
    corpus=px.Inferences(corpus_df.reset_index(drop=True), corpus_schema, "corpus"),
)

## Conclusion

We have explored how to build and evaluate a RAG pipeline using LlamaIndex and Phoenix, with a specific focus on evaluating the retrieval system and generated responses within the pipelines. 

Phoenix offers a variety of other evaluations that can be used to assess the performance of your LLM Application. For more details, see the [LLM Evals](https://docs.arize.com/phoenix/llm-evals/llm-evals) documentation.